In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# load data

cross = pd.read_csv('20201125_cross.csv', low_memory=False)
cross = cross.drop([0,1])
cross = cross.reset_index()

cross = cross.replace('AA', -1)
cross = cross.replace('AB', 0)
cross = cross.replace('BB', 1)
cross = cross.replace('-', np.nan)

cross.GENERATION = ['F'+str(i) for i in cross.GENERATION]
cross.DAM = ['D'+str(i) for i in cross.DAM]

cross = cross.loc[cross.BW8.isna()==False]
cross.BW8 = cross.BW8.astype(float)
cross.SEX = cross.SEX.astype(float)


cnew = []
for i in cross.columns:
    if '-' in i:
        ni = 'mrk_'+'_'.join(i.split('-'))
    else:
        ni = i
    cnew.append(ni)
    
cross.columns = cnew


cross_markers = [i for i in cross.columns if i.startswith('mrk')]


In [4]:
#cross

# building a simple ols instead of the linear model

In [5]:
data = cross
pheno = 'BW8'
md = smf.ols("{pheno} ~ SEX + GENERATION".format(pheno=pheno), data=data)
mdf = md.fit()

In [6]:
mdf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BW8   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     138.0
Date:                Tue, 01 Dec 2020   Prob (F-statistic):          1.35e-294
Time:                        13:24:13   Log-Likelihood:                -19734.
No. Observations:                3073   AIC:                         3.950e+04
Df Residuals:                    3059   BIC:                         3.958e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           575.6627     19.446     29.603      0.000     537.534     613.792
GENERATION[T.F11]   -60.9723     25.030     -2.436      0.015    -110.050     -11.895
GENERATION[T.F13]   -16.6544     25.759     -0.647      0.518     -67.160      33.852
GENERATION[T.F14]    68.3692     32.651      2.094      0.036       4.348     132.390
GENERATION[T.F15]   -52.2795     19.950     -2.620      0.009     -91.397     -13.162
GENERATION[T.F2]    -17.2503     19.905     -0.867      0.386     -56.279      21.778
GENERATION[T.F3]     63.2823     20.565      3.077      0.002      22.960     103.604
GENERATION[T.F4]    -53.5611     23.864     -2.244      0.025    -100.352      -6.771
GENERATION[T.F5]      6.2125     23.719      0.262      0.793     -40.294      52.719
GENERATION[T.F6]    122.4939     24.972      4.905      0.000      73.531     171.457
GENERATION[T.F7]     43.9834     30.322      1.451      0.147     -15.469     103.436
GENERATION[T.F8]   -273.9595     21.161    -12.947      0.000    -315.450    -232.469
GENERATION[T.F9]     81.2095     28.561      2.843      0.004      25.209     137.210
SEX                 141.1413      5.431     25.990      0.000     130.493     151.789
==============================================================================
Omnibus:                       22.307   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.484
Skew:                          -0.163   Prob(JB):                     4.82e-06
Kurtosis:                       3.291   Cond. No.                         32.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
mdf.resid

0      -459.553657
1       -11.553657
2       142.587656
3       -23.412344
4        12.587656
           ...    
3130   -132.945015
3131    117.913672
3132    127.913672
3133   -210.086328
3134    -40.945015
Length: 3073, dtype: float64

In [8]:
residuals = pd.DataFrame([cross['id.1'], mdf.resid]).T

In [9]:
residuals.columns = ['id', 'sex_gen_resid']
residuals.to_csv('20201201_residuals_precorrected_for_sex_and_generation_as_fixed_effects.csv')